In [3]:
import pandas as pd
import datetime
import numpy as np
import time

pd.options.mode.chained_assignment = None  # default='warn'

In [10]:
d_list=    [ '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
           '180711', '180712', '180713', '180714', '180715',\
           '180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725',\
           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804',\
           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822', \
            '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
           '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
           '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
            '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
           '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
           '181016', '181017', '181018', '181019', '181020', '181021',\
            '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029', '181030', \
           '181031', '181101', '181102', '181103', '181104']

day_all= ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
           '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
           '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
           '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
           '180711', '180712', '180713', '180714', '180715',\
           '180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725',\
           '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804',\
           '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
           '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822', \
            '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
           '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
           '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
            '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
           '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
           '181016', '181017', '181018', '181019', '181020', '181021',\
            '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029', '181030', \
           '181031', '181101', '181102', '181103', '181104', '181105']

### 统计事件表中每个状态，每个作业每天的状态变化计数
训练的时候 ,只能用前几天的特征

In [11]:
#只对第二天4点前结束的作业有效，训练也应取4点前结束的训练

flag = 0
# for day in d_list:
for day in d_list:
    
    def get_clc_job_begain_time(x):
        if x[0] in clc_jobhead:
            return '20'+day+'000000'
        else:
            return x[1]


    day_i  = day_all.index(day)

    day_after = int(day_all[day_i+1])
    day_after_end = int('20'+day_all[day_i+1] +'040000')
    
    
    #-------------------------------------------------------------------------------------
    # 实例化时今天， 结束在今天和第二天的所有作业--ajob
    
    ajob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob_0\ajob_%s_0.csv'%day,low_memory=False)
    ajob = ajob[ajob.end_time<= day_after_end]

    
    #print(ajob.shape)
    
    
    # ajob2 在第二天结束的作业
    ajob2 = ajob[ajob.v_date == day_after]
    ajob2 = ajob2[['order_id','job_name','server']]
    #print(ajob2.shape)

    ajob = ajob[['order_id','job_name','end_time']]
    
    
    #-------------------------------------------------------------------------------------
    #取事件表 并处理
    if flag == 0:
        evet1 = pd.read_csv(r'E:\jupyter\CTM\data_src\evet\evet_%s.csv'%day)
        evet1 = evet1[['order_id','evt_time','new_value', 'old_value','server']]
        evet1 = evet1.dropna()
        evet1 = evet1.drop_duplicates(keep='first')

        evet1['server'] = evet1['server'].apply(str)
        evet1['order_id'] = evet1[['order_id','server']].apply(lambda x:x[0]+x[1],axis=1)
        flag = 1
    else:
        evet1 = evet2
        
    evet1 = evet1.merge(ajob,how='left',on='order_id')
    evet1 = evet1[evet1.job_name.notnull()]
    #print(evet1.shape)

    #--------------------------------------------------------------------------------------
    
    if day == d_list[-1]:
        pass
    else:
        evet2 = pd.read_csv(r'E:\jupyter\CTM\data_src\evet\evet_%s.csv'%str(day_after))   
        evet2 = evet2[['order_id','evt_time','new_value', 'old_value','server']]
        evet2 = evet2.dropna()
        evet2 = evet2.drop_duplicates(keep='first')

        evet2['server'] = evet2['server'].apply(str)
        evet2['order_id'] = evet2[['order_id','server']].apply(lambda x:x[0]+x[1],axis=1)

    if ajob2.shape[0]<1:
        print('next_day not find')
    else:
        evet2_merge = evet2.merge(ajob2,how='left',on='order_id')
        evet2_merge = evet2_merge[evet2_merge.job_name.notnull()]
        evet1 = pd.concat([evet1,evet2_merge], ignore_index=True)

    evet_groupby = evet1.groupby('order_id').order_id.count()
    evet_groupby =evet_groupby.to_frame(name='all_count')
    evet_groupby['order_id'] = evet_groupby.index
    
    evet_groupby = evet_groupby.merge(ajob[['order_id','job_name']],on='order_id',how='left')
    
    for col in ['new_value','old_value']:
        for state in [1,2,4,8,32,2048]:
            state_groupby = evet1[evet1[col]==state].groupby('order_id').order_id.count()
            state_groupby =state_groupby.to_frame(name='%s_%d'%(col,state))
            state_groupby['order_id'] = state_groupby.index
            
            evet_groupby = evet_groupby.merge(state_groupby,how='left',on='order_id')
            
            evet_groupby.fillna(0,inplace=True)
            del state_groupby

    evet_groupby = evet_groupby.round(3)
    del ajob
    evet_groupby.to_csv(r'E:\jupyter\CTM\data_processed\evet_count\evet_count_%s.csv'%day,index=False)
    print(day,end=' ')

180603 180604 180605 180606 180607 180608 180609 180610 180611 180612 180613 180614 180615 180616 180617 180618 180619 180620 180621 180622 180623 180624 180625 180626 180627 180628 180629 180630 180701 180702 180703 180704 180705 180706 180707 180708 180709 180710 180711 180712 180713 180714 next_day not find
180715 180716 180717 180718 180719 180720 180721 180722 180723 180724 180725 180726 180727 180728 180729 180730 180731 180801 180802 180803 180804 180805 180806 180807 180808 180809 180810 180811 180812 180813 180814 180815 180816 180817 180818 180819 180820 180821 next_day not find
180822 180828 180829 180830 180831 180901 180902 180903 180904 180905 180906 180907 180908 180909 180910 180911 180912 180913 180914 180915 180916 180917 180918 180919 180920 180921 180922 180923 180924 180925 180926 180927 180928 180929 180930 181001 181002 181003 181004 181005 181006 181007 181008 181009 181010 181011 181012 181013 181014 181015 181016 181017 181018 181019 181020 next_day not find
1

180602 

### 统计0-4点作业状态变化计数，按作业名前缀分组 
训练时，每个都可以用当天的特征

In [17]:
# day_all = ['180602', '180603', '180604', '180605', '180606', '180607', '180608', '180609', '180610',\
#            '180611', '180612', '180613', '180614', '180615', '180616', '180617', '180618', '180619', '180620',\
#            '180621', '180622', '180623', '180624', '180625', '180626', '180627', '180628', '180629', '180630',\
#            '180701', '180702', '180703', '180704', '180705', '180706', '180707', '180708', '180709', '180710', \
#            '180711', '180712', '180713', '180714', '180715',\
#            '180716', '180717', '180718', '180719', '180720', '180721', '180722', '180723', '180724', '180725',\
#            '180726', '180727', '180728', '180729', '180730', '180731', '180801', '180802', '180803', '180804',\
#            '180805', '180806', '180807', '180808', '180809', '180810', '180811', '180812', '180813', '180814',\
#            '180815', '180816', '180817', '180818', '180819', '180820', '180821', '180822', \
#            '180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
#            '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
#            '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
#             '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
#            '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
#            '181016', '181017', '181018', '181019', '181020', '181021',\
#            '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029', '181030', \
#            '181031', '181101', '181102', '181103', '181104', '181105', '181106', '181107', '181108', '181109',\
#            '181110', '181111']
day_all = ['180828', '180829', '180830', '180831', '180901', '180902', '180903', '180904', '180905',\
           '180906', '180907', '180908', '180909', '180910', '180911', '180912', '180913', '180914', '180915',\
           '180916', '180917', '180918', '180919', '180920', '180921', '180922', '180923', '180924', '180925', \
            '180926', '180927', '180928', '180929', '180930', '181001', '181002', '181003', '181004', '181005', \
           '181006', '181007', '181008', '181009', '181010', '181011', '181012', '181013', '181014', '181015', \
           '181016', '181017', '181018', '181019', '181020', '181021',\
           '181022', '181023', '181024', '181025', '181026', '181027', '181028', '181029', '181030', \
           '181031', '181101', '181102', '181103', '181104', '181105', '181106', '181107', '181108', '181109',\
           '181110', '181111']

In [24]:
prefixs_2 = ('RT_RST','RF_RST','SYS_FS','FW_LD','SYS_FW','EXZ_T','EX_P','FW_TB','RU_P',\
             'FW_TS','EXZ_P','EX_A','LD_TDW','STG_V','EXZ_A','STG_C','PDM_P','RU_HCDC','EXM_P',\
             'PDM_C','PDM_F','EX_T','NDS_P','NDS_C','TB_A','EXM_T','TS_HEES','EL_DXP','EXM_A','BRTL_C')

cls = []
for cl in ['old_','new_']:
    for state in (1,2,4,8,16,32,2048):
        for job_prefix in prefixs_2:
            cls.append(cl+str(state)+'_'+job_prefix)
allday_list =[]
allday_index = []

def event_0_4_finished_cnt(day):
    #day = '181028'
    everyday_list = []

    df = pd.read_csv(r'E:\jupyter\CTM\data_src\evet\evet_%s.csv'%day,low_memory=False)
    #2018-10-24 01:10:54
    #print(df.shape)
    endt_th = '20'+ day[:2]+'-'+day[2:4]+'-'+day[4:] +' 04:00:00'
    df = df[df['evt_time']<=endt_th]    
    #print(df.shape)
    #df['evt_time'] = df['evt_time'].apply(lambda x: int(''.join(''.join((''.join(x.split('-'))).split(' ')).split(':'))))

    #endt_th = int('20'+ day +'040000')
    #df = df[df['evt_time']<endt_th]
    df['server'] = df['server'].apply(str)
    df['order_id'] = df[['order_id','server']].apply(lambda x:str(x[0])+str(x[1]),axis=1)

    djob = pd.read_csv(r'E:\jupyter\CTM\data_src\ajob_0\ajob_%s_0.csv'%day,low_memory=False)
    df = df.merge(djob,how='left',on='order_id')
    df = df[df.job_name.notnull()]

    df['job_prefix'] = df['job_name'].apply(lambda x:'_'.join(x.split("_")[0:2]))

    for cl in ['old_value','new_value']:
        for state in (1,2,4,8,16,32,2048):
            for job_prefix in prefixs_2:
                everyday_list.append(df[(df[cl]==state)&(df['job_prefix']==job_prefix)].order_id.count())
    return everyday_list


In [25]:
for day in day_all:
    print(day,end=' ')
    allday_list.append(event_0_4_finished_cnt(day))
    allday_index.append(day)
    
fnish_at_0_4_countdf = pd.DataFrame(allday_list,columns=cls)
fnish_at_0_4_countdf.index = allday_index
print(fnish_at_0_4_countdf.shape)

180828 180829 180830 180831 180901 180902 180903 180904 180905 180906 180907 180908 180909 180910 180911 180912 180913 180914 180915 180916 180917 180918 180919 180920 180921 180922 180923 180924 180925 180926 180927 180928 180929 180930 181001 181002 181003 181004 181005 181006 181007 181008 181009 181010 181011 181012 181013 181014 181015 181016 181017 181018 181019 181020 181021 181022 181023 181024 181025 181026 181027 181028 181029 181030 181031 181101 181102 181103 181104 181105 181106 181107 181108 181109 181110 181111 (76, 420)


In [26]:
fnish_at_0_4_countdf.to_csv(r'E:\jupyter\CTM\data_processed\evet_count_byfolder\evet_count_byfolder.csv',index=True)

In [14]:
fnish_at_0_4_countdf = pd.DataFrame(allday_list,columns=cls)
fnish_at_0_4_countdf.index = allday_index
print(fnish_at_0_4_countdf.shape)

(47, 420)


In [16]:
len(day_all)

76

In [20]:
day[:2]+'-'+day[2:4]+'-'+day[4:]

'18-08-28'